In [74]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
# ================== Dataset ==================
class CarlaDataset(Dataset):
    def __init__(self, images, angles, signals, transform=None):
        self.images = images
        self.angles = angles
        self.signals = signals
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        angle = self.angles[idx]
        signal = self.signals[idx]

        # only for straight cases:
        if signal == 0.0:
            if abs(angle) <= 0.05:
                angle = 0.0
            elif angle > 0.05:
                angle = angle * 0.45
            # if angle < -0.05, leave it unchanged

        if self.transform:
            img = self.transform(img)

        return (
            torch.tensor(img, dtype=torch.float32),
            torch.tensor(signal).unsqueeze(0),
            torch.tensor(angle, dtype=torch.float32)
        )


In [76]:
# ================== Model ==================
class SteeringModel(nn.Module):
    def __init__(self):
        super(SteeringModel, self).__init__()
        resnet = models.resnet18(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad = False
        for param in resnet.layer4.parameters():
            param.requires_grad = True

        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])  # (B, 512, 1, 1)

        self.signal_fc = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU()
        )

        self.fc_left = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.fc_right = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.fc_straight = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, image, signal):
        x = self.cnn_backbone(image)
        x = x.view(x.size(0), -1)
        s = self.signal_fc(signal)
        combined = torch.cat([x, s], dim=1)

        preds = torch.zeros(image.size(0), 1, device=image.device)
        left_mask = (signal.squeeze(1) == -1)
        right_mask = (signal.squeeze(1) == 1)
        straight_mask = (signal.squeeze(1) == 0)

        if left_mask.any():
            preds[left_mask] = self.fc_left(combined[left_mask])
        if right_mask.any():
            preds[right_mask] = self.fc_right(combined[right_mask])
        if straight_mask.any():
            preds[straight_mask] = self.fc_straight(combined[straight_mask])

        return preds.squeeze(1)

In [77]:
# ================== Loss ==================
class DirectionalWeightedMSE(nn.Module):
    def __init__(self, left_weight=0.0, right_weight=0.0, straight_weight=1.0):
        super().__init__()
        self.left_weight = left_weight
        self.right_weight = right_weight
        self.straight_weight = straight_weight

    def forward(self, preds, targets, signals):
        weights = torch.ones_like(targets)
        weights[signals.squeeze() == -1] = self.left_weight
        weights[signals.squeeze() == 1] = self.right_weight
        weights[signals.squeeze() == 0] = self.straight_weight
        loss = weights * (preds - targets) ** 2
        return loss.mean()

In [78]:
# ================== Load Data ==================
images = np.load('../output/std_images.npy')         # (N, 3, 224, 224)
angles = np.load('../output/std_angles.npy').astype(np.float32)
signals = np.load('../output/std_turn_signals.npy').astype(np.float32)

train_idx, val_idx = train_test_split(
    np.arange(len(images)),
    test_size=0.2,
    shuffle=False  # No shuffling for time series data
)

train_data = (images[train_idx], angles[train_idx], signals[train_idx])
val_data   = (images[val_idx],   angles[val_idx],   signals[val_idx])

In [79]:
# ================== Transforms ==================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225]
    )
])

In [80]:

train_loader = DataLoader(
    CarlaDataset(*train_data, transform=transform),
    batch_size=32, shuffle=False
)
val_loader = DataLoader(
    CarlaDataset(*val_data, transform=transform),
    batch_size=32, shuffle=False
)

In [86]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

# — assume CarlaDataset and SteeringModel are defined above —

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = SteeringModel().to(device)

# where to save / load
os.makedirs("../models", exist_ok=True)
checkpoint_path = "../models/RDModel_checkpoint.pth"

# — load checkpoint if it exists —
if os.path.isfile(checkpoint_path):
    ckpt = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ckpt['model_state'])
    start_epoch   = ckpt['epoch'] + 1
    best_val_loss = ckpt['best_val']
    print(f"Resuming from epoch {ckpt['epoch']} (best_val={best_val_loss:.4f})")
else:
    print("No checkpoint found, starting from scratch")
    start_epoch   = 1
    best_val_loss = float('inf')

# — define per-component learning rates —
lr_backbone = 1e-3    # fine-tune ResNet backbone & signal encoder
lr_left     = 1e-3    # left-turn head
lr_right    = 1e-3    # right-turn head
lr_straight = 1e-3    # straight-ahead head

# — build optimizer with separate param groups —
optimizer = torch.optim.Adam([
    { "params": model.cnn_backbone.parameters(), "lr": lr_backbone },
    { "params": model.signal_fc.parameters(),    "lr": lr_backbone },
    { "params": model.fc_left.parameters(),      "lr": lr_left    },
    { "params": model.fc_right.parameters(),     "lr": lr_right   },
    { "params": model.fc_straight.parameters(),  "lr": lr_straight},
], weight_decay=1e-5)

criterion = nn.MSELoss()

# — prepare data loaders —
images  = np.load('../output/all_images.npy')
angles  = np.load('../output/all_angles.npy').astype(np.float32)
signals = np.load('../output/all_turn_signals.npy').astype(np.float32)

idxs      = np.arange(len(images))
train_idx, val_idx = train_test_split(idxs, test_size=0.2, shuffle=False)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_loader = DataLoader(
    CarlaDataset(images[train_idx], angles[train_idx], signals[train_idx], transform),
    batch_size=32, shuffle=False
)
val_loader = DataLoader(
    CarlaDataset(images[val_idx],   angles[val_idx],   signals[val_idx],   transform),
    batch_size=32, shuffle=False
)

# — training loop —
num_epochs = 5
for epoch in range(start_epoch, num_epochs + 1):
    # Training
    model.train()
    total_train = 0.0
    for imgs, sigs, angs in train_loader:
        imgs, sigs, angs = imgs.to(device), sigs.to(device), angs.to(device)
        optimizer.zero_grad()
        preds = model(imgs, sigs)
        loss  = criterion(preds, angs)
        loss.backward()
        optimizer.step()
        total_train += loss.item()
    train_loss = total_train / len(train_loader)

    # Validation
    model.eval()
    total_val = 0.0
    with torch.no_grad():
        for imgs, sigs, angs in val_loader:
            imgs, sigs, angs = imgs.to(device), sigs.to(device), angs.to(device)
            total_val += criterion(model(imgs, sigs), angs).item()
    val_loss = total_val / len(val_loader)

    print(f"Epoch {epoch} | Train {train_loss:.4f} | Val {val_loss:.4f}")

    # Checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch':       epoch,
            'model_state': model.state_dict(),
            'opt_state':   optimizer.state_dict(),
            'best_val':    best_val_loss,
        }, checkpoint_path)
        print(f"  → Checkpoint saved (val {best_val_loss:.4f})")


No checkpoint found, starting from scratch


C:\Users\Pravin\AppData\Local\Temp\ipykernel_11564\3279276879.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(img, dtype=torch.float32),


Epoch 1 | Train 0.0092 | Val 0.0059
  → Checkpoint saved (val 0.0059)
Epoch 2 | Train 0.0035 | Val 0.0036
  → Checkpoint saved (val 0.0036)
Epoch 3 | Train 0.0029 | Val 0.0047
Epoch 4 | Train 0.0027 | Val 0.0178
Epoch 5 | Train 0.0028 | Val 0.0076


In [88]:
torch.save(
    model.state_dict(),
    "../models/RDModel_finetuned.pth"
)
print("Saved fine-tuned weights to ../models/RDModel_finetuned.pth")

Saved fine-tuned weights to ../models/RDModel_finetuned.pth
